In [1]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from datetime import datetime
from config import weather_api_key
from scipy.stats import linregress
import matplotlib.pyplot as plt

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [2]:
# Create a set of 1,500 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

coordinates = list(lat_lngs)

# Create a list for holding the cities.

# Get the nearest city using the citipy module.
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

600

In [3]:
# Perform an API call with the OpenWeatherMap.

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")


city_data = []
record_count = 1
set_count = 1
    
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_descr = city_weather["weather"][0]["main"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        if city_weather["weather"][0]["main"] == "Rain":
            city_rain = city_weather["rain"]["3h"]
        else:
            city_rain = 0
       
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        if city_weather["weather"][0]["main"] == "Snow":
            city_snow = city_weather["snow"]["3h"]
        else:
            city_snow = 0

        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_descr,
                          "Rain inches": city_rain,
                          "Snow inches": city_snow,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print(f"City not found. Skipping... {city} ")
        pass
            
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
print(len(city_data))

Beginning Data Retrieval     
-----------------------------
City not found. Skipping... jamestown 
City not found. Skipping... grand river south east 
City not found. Skipping... attawapiskat 
City not found. Skipping... hihifo 
City not found. Skipping... yellowknife 
City not found. Skipping... severnyy 
City not found. Skipping... umzimvubu 
City not found. Skipping... albany 
City not found. Skipping... sentyabrskiy 
City not found. Skipping... grindavik 
City not found. Skipping... kazalinsk 
City not found. Skipping... nizhneyansk 
City not found. Skipping... chapais 
City not found. Skipping... lolua 
City not found. Skipping... taolanaro 
City not found. Skipping... belushya guba 
City not found. Skipping... sitka 
City not found. Skipping... vaitupu 
City not found. Skipping... marcona 
City not found. Skipping... high level 
City not found. Skipping... ikole 
City not found. Skipping... palabuhanratu 
City not found. Skipping... sorland 
City not found. Skipping... albanel 
C

In [4]:
city_data_df = pd.DataFrame(city_data)
new_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description","Rain inches","Snow inches"]
city_data_df = city_data_df[new_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches,Snow inches
0,Powell River,CA,2020-02-28 05:25:03,49.85,-124.54,46.40,87,90,9.17,Clouds,0.00,0.0
1,Basco,PH,2020-02-28 05:28:03,20.45,121.97,75.25,82,40,18.88,Rain,1.13,0.0
2,Nara,JP,2020-02-28 05:28:03,34.69,135.80,51.01,39,75,2.24,Clouds,0.00,0.0
3,Narsaq,GL,2020-02-28 05:28:04,60.92,-46.05,6.80,42,0,3.36,Clear,0.00,0.0
4,Kaitangata,NZ,2020-02-28 05:28:04,-46.28,169.85,75.00,69,79,4.00,Clouds,0.00,0.0
5,Jishui,CN,2020-02-28 05:28:05,33.73,115.40,50.13,74,98,1.12,Clouds,0.00,0.0
6,Maragogi,BR,2020-02-28 05:28:05,-9.01,-35.22,82.02,79,29,11.52,Clouds,0.00,0.0
7,Smithers,CA,2020-02-28 05:27:31,54.78,-127.17,35.60,80,90,12.75,Clouds,0.00,0.0
8,Idah,NG,2020-02-28 05:28:06,7.11,6.73,79.56,77,93,1.88,Clouds,0.00,0.0
9,Hasaki,JP,2020-02-28 05:28:06,35.73,140.83,57.00,37,40,4.70,Clouds,0.00,0.0


In [5]:
# Export the output file (CSV)
output_data_file = "Challenge/WeatherPy_challenge.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")